In [1]:
from bot import Bot
from game_config import Config, Blackjack
from shoe import Shoe
from tqdm import tqdm_notebook as tqdm

from collections import Counter
import json
import pickle

In [2]:
def simulate(n_loop: int = 1000, faceup_target: str = None, hand_target: list = None, 
            action_target: str = None): # n_player = 1
    pbar = tqdm(total = n_loop)
    
    bet_amount = 1

    config = Config() # Define your configuration game here. Check game_config.py for reference

    blackjack = Blackjack(config = config, player = [Bot(bankroll = 1_000_000, name = 'Player 1')],
                          shoe = Shoe(deck_quant = config.n_deck))
    card_quant = len(blackjack.shoe.shoe) # Total cards in the shoe

    loop = 0
    while loop < n_loop:
        if len(blackjack.shoe.shoe) < int(card_quant*blackjack.config.reset_shoe):
            blackjack.shoe.shuffle(reset=True)
            
        game = blackjack.run()

        dealer_hand = game.dealer
        bot = game.players[0]

        faceup_value = str(dealer_hand.get_faceup_card().get_score()[0])
        bot_value = sorted([card.get_score()[0] for card in bot.hand], reverse=True)

        if faceup_target:
            if faceup_target != faceup_value:
                continue
                
        if hand_target:
            if sorted(hand_target, reverse=True) != bot_value:
                continue

        # Processing block
        for bot_hand in [bot]: # Player-scanning Block
            hand_process = [bot_hand]
            hand_count = 0

            while len(hand_process) != 0: # All-hands loop Block
                hand_in_play = hand_process[0]
                _count = 0

                quest = game.question(hand_in_play, _count, insurance_offer = False)
                if hand_count != 0:
                    quest = 'Hand {} --- {}'.format(hand_count,quest)

                while quest != None: # Response Block
                    # Bot action
                    if action_target and hand_count == 0 and _count == 0:
                        action = action_target
                        if action_target == 'split' and hand_in_play.split_check() == False:
                            break
                    else:
                        action = hand_in_play.player.play(faceup_card = game.dealer.get_faceup_card(), hand = hand_in_play, question = quest)
                    _count += 1

                    if action == 'x2':
                        hand_in_play.x2(blackjack.shoe)
#                         print(hand_in_play.get_hand())
                    elif action == 'hit':
                        hand_in_play.hit(blackjack.shoe)
#                         print(hand_in_play.get_hand())
                    elif action == 'stand':
                        hand_in_play.stand()
                    elif action == 'split':
                        hand_1, hand_2 = hand_in_play.split(blackjack.shoe)
                        hand_process += [hand_1, hand_2]
                        # hand_delete = True
                        break

                    quest = game.question(hand_in_play, _count)
#                     if quest != None:
# #                         print('Quest: {}'.format(quest))

                hand_process.pop(0)

                hand_count += 1
        
        game.dealer_action(blackjack.shoe) 
#         print('-----------------------------------------------')
#         print(hand_print(dealer_hand, player=False))

        for bot_hand in [bot]: # Player-scanning Score block
#             print('Bot {} ------ Results: '.format(bot_hand.player.name))
            hands = game.get_hands(player_name = bot_hand.player.name)
            for hand in hands:
#                 print(hand_print(hand))
                game.process_result(hand, bet_amount, printout=False)

        blackjack.log(game)

        loop += 1
        
        pbar.update(1)

    return blackjack.logs
    

In [3]:
# log = simulate(n_loop = 100, faceup_target = '1', hand_target = [4,5], action_target = 'hit')

In [3]:
with open('data_raw_3','rb') as file:
    data = pickle.load(file)

In [4]:
last_target = max(data.keys())
last_var1 = max(data[last_target].keys())
last_var2 = max(data[last_target][last_var1].keys())
last_act = list(data[last_target][last_var1][last_var2].keys())[-1]

In [5]:
def next_loop(last_target: int, last_var1: int, last_var2: int, last_act: str):
    
    def next_act(act: str):
        act_list = ['hit','stand','x2','split']
        next_idx = act_list.index(act) + 1
        next_idx = 0 if next_idx == len(act_list) else next_idx
        return act_list[next_idx]
    
    n_act = next_act(last_act)

    if n_act == 'hit':
        if last_var2 == 10:
            if last_var1 == 10:
                n_var1 = n_var2 = 1
                if last_target == 10:
                    n_target, n_var1, n_var2, n_act = None
                else:
                    n_target = last_target + 1
            else:
                n_var2 = n_var1 = last_var1 + 1
                n_target = last_target
        else:
            n_var2 = last_var2 + 1
            n_target, n_var1 = last_target, last_var1
    else:
        n_target, n_var1, n_var2 = last_target, last_var1, last_var2
        
    return n_target, n_var1, n_var2, n_act
                

In [6]:
next_loop(last_target, last_var1, last_var2, last_act)

(3, 1, 4, 'split')

In [7]:
data = {}

In [8]:
n_target, n_var1, n_var2, n_act = next_loop(last_target, last_var1, last_var2, last_act)

while n_target != None:
    if data.get(n_target) == None:
        data[n_target] = {}
        print('Create Target %s' %n_target)
        
    if data.get(n_target).get(n_var1) == None:
        data[n_target][n_var1] = {}
        print('Create Var1 %s' %n_var1)
        
    if data.get(n_target).get(n_var1).get(n_var2) == None:
        data[n_target][n_var1][n_var2] = {}
        print('Create Var2 %s' %n_var2)
        
    log = simulate(n_loop = 10000, faceup_target = str(n_target), hand_target = [n_var1, n_var2], 
                   action_target = n_act )

    data[n_target][n_var1][n_var2][n_act] = log
    
    n_target, n_var1, n_var2, n_act = next_loop(n_target, n_var1, n_var2, n_act)

Create Target 3
Create Var1 1
Create Var2 4


/Users/duongnt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Create Var2 5


Create Var2 6


Create Var2 7


...

Create Var2 10


Create Var1 6
Create Var2 6


Create Var2 7


Create Var2 8


KeyboardInterrupt: 

In [9]:
with open('data_raw_4','wb') as file:
    pickle.dump(data,file)

In [4]:
# for target in range(n_target,11):
    
#     if data.get(n_target) == None:
#         data[n_target] = {}

#     for var1 in range(n_var1,11):
        
#         if data.get(n_target).get(n_var1) == None:
#             data[n_target][n_var1] = {}
            
#         for var2 in range(n_var2,11):
#             if var2 >= var1:
                
#                 if data.get(n_target).get(n_var1).get(n_var2) == None:
#                     data[n_target][n_var1][n_var2] = {}
                    
#                 for act in ['hit','stand','x2','split']:
                
#                     log = simulate(n_loop = 10000, faceup_target = str(target), hand_target = [var1, var2], action_target = act )
#                     data[target][var1][var2][act] = log

/Users/duongnt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [5]:
with open('data_raw','wb') as file:
    pickle.dump(data,file)

In [6]:
with open('data_raw','rb') as file:
    test = pickle.load(file)

In [7]:
test

{1: {1: {1: {'hit': [<game_config.Round_play at 0x10f48f6a0>,
     ...]}}}}

In [9]:
r = Counter([v.players[0].result for v in log])

In [10]:
win = r['win']
lose = r['lose']
tie = r['tie']

In [11]:
(win-lose)/(win+lose+tie)

0.673

In [12]:
r

Counter({'win': 673, 'tie': 327})

In [9]:
def simulate(n_player: int = 1, n_loop: int = 1000, faceup_target: str = None, hand_target: list = None):
    bet_amount = 1
    bot_signup = []
    for i in range(n_player):
        bot_signup.append(Bot(bankroll = 1_000_000, name = 'Player {}'.format(i+1)))

    config = Config() # Define your configuration game here. Check game_config.py for reference

    blackjack = Blackjack(config = config, player = bot_signup, shoe = Shoe(deck_quant = config.n_deck))
    card_quant = len(blackjack.shoe.shoe) # Total cards in the shoe

    loop = 0
    while loop < n_loop:
        game = blackjack.run()

        dealer_hand = game.dealer
        bots = game.players

        faceup_value = str(dealer_hand.get_faceup_card().get_score()[0])
        bots_value = [sorted([card.get_score()[0] for card in bot.hand], reverse=True) for bot in bots]

        if faceup_target:
            if faceup_target != faceup_value:
                continue
                
        if hand_target:
            if sorted(hand_target, reverse=True) not in bots_value:
                continue
            else:
                indices = [i for i,v in enumerate(bots_value) if v == sorted(hand_target, reverse = True)]
        # Processing block
        for bot_hand in bots: # Player-scanning Block
            hand_process = [bot_hand]
            hand_count = 0

            while len(hand_process) != 0: # All-hands loop Block
                hand_in_play = hand_process[0]
                _count = 0

                quest = game.question(hand_in_play, _count)
                if hand_count != 0:
                    quest = 'Hand {} --- {}'.format(hand_count,quest)

				
				print('Bot Hand: {}'.format(hand_in_play.get_hand()))
				print('⍰ Quest: {}'.format(quest))
				while quest != None: # Response Block
					# Bot action
					action = hand_in_play.player.play(faceup_card = game.dealer.get_faceup_card(), hand = hand_in_play, question = quest)
					_count += 1
					print('--> {}'.format(action))

					if action == 'x2':
						hand_in_play.x2(blackjack.shoe)
						print(hand_in_play.get_hand())
					elif action == 'hit':
						hand_in_play.hit(blackjack.shoe)
						print(hand_in_play.get_hand())
					elif action == 'stand':
						hand_in_play.stand()
					elif action == 'split':
						hand_1, hand_2 = hand_in_play.split(blackjack.shoe)
						hand_process += [hand_1, hand_2]
						# hand_delete = True
						break

					quest = game.question(hand_in_play, _count)
					if quest != None:
						print('Quest: {}'.format(quest))

				hand_process.pop(0)

				hand_count += 1

		game.dealer_action(blackjack.shoe) 
		print('-----------------------------------------------')
		print(hand_print(dealer_hand, player=False))

		for bot_hand in bots: # Player-scanning Score block
			print('Bot {} ------ Results: '.format(bot_hand.player.name))
			hands = game.get_hands(player_name = bot_hand.player.name)
			for hand in hands:
				print(hand_print(hand))
				game.process_result(hand, bet_amount)

		blackjack.log(game)



        if len(blackjack.shoe.shoe) < int(card_quant*blackjack.config.reset_shoe):
            blackjack.shoe.shuffle(reset=True)


   